# Multi-State UHC Rates Comparison

Comparing all UHC state/plan extractions to identify:
1. Duplication patterns across states/plans
2. Subset relationships (which files contain others)
3. Master file identification (largest/most comprehensive)
4. Unique data per file

**Files being compared:**
- IL Choice Plus (Dec 19, 2025)
- IL Core POS (Dec 27, 2025)
- NC Choice Plus (Jan 2, 2026)
- NY Choice EPO (Dec 28, 2025)
- NY Choice Plus (Dec 29, 2025)
- GA Charter HMO (Dec 30, 2025)
- GA Choice HMO (Jan 1, 2026)


In [1]:
import pandas as pd
import pyarrow.parquet as pq
from pathlib import Path
from collections import defaultdict

# Define all UHC files to compare
uhc_files = {
    "IL Choice Plus": "output/il_uhc/rates_il_uhc_choice_plus_20251219_194236.parquet",
    "IL Core POS": "output/il_uhc/rates_il_uhc_core_pos_20251227_121655.parquet",
    "NC Choice Plus": "output/nc_uhc/rates_nc_uhc_choice_plus_20260102_112128.parquet",
    "NY Choice EPO": "output/ny_uhc/rates_ny_uhc_choice_epo_20251228_112232.parquet",
    "NY Choice Plus": "output/ny_uhc/rates_ny_uhc_choice_plus_20251229_160021.parquet",
    "GA Charter HMO": "output/ga_uhc/rates_ga_uhc_charter_hmo_20251230_113902.parquet",
    "GA Choice HMO": "output/ga_uhc/rates_ga_uhc_choice_hmo_20260101_182048.parquet",
}

# Verify files exist and get sizes
print("📁 File Verification:")
print("="*80)
existing_files = {}
for name, path in uhc_files.items():
    file_path = Path(path)
    if file_path.exists():
        size_gb = file_path.stat().st_size / (1024**3)
        existing_files[name] = path
        print(f"✅ {name:20s} {size_gb:5.2f} GB")
    else:
        print(f"❌ {name:20s} NOT FOUND: {path}")

print(f"\n📊 Total files to compare: {len(existing_files)}")


📁 File Verification:
✅ IL Choice Plus        1.81 GB
✅ IL Core POS           1.70 GB
✅ NC Choice Plus        1.76 GB
✅ NY Choice EPO         1.75 GB
✅ NY Choice Plus        1.75 GB
✅ GA Charter HMO        1.61 GB
✅ GA Choice HMO         1.75 GB

📊 Total files to compare: 7


In [2]:
# Memory-efficient streaming: Extract unique keys from ONE file at a time
# Save results to disk to avoid memory issues with multiple large sets

import pickle
from pathlib import Path

def extract_unique_keys(pq_file, label):
    """Extract unique provider IDs, rate keys, and billing codes from ENTIRE file via streaming"""
    provider_ids = set()
    rate_keys = set()
    billing_codes = set()
    
    total_groups = pq_file.num_row_groups
    columns = ['provider_reference_id', 'billing_code', 'billing_code_type', 'negotiated_rate']
    
    print(f"\n📥 Streaming {label} (processing ALL {total_groups} row groups)...")
    for i in range(total_groups):
        # Read one row group at a time (memory-efficient)
        table = pq_file.read_row_group(i, columns=columns)
        df_chunk = table.to_pandas()
        
        # Extract unique values (updates sets, not storing full dataframe)
        provider_ids.update(df_chunk['provider_reference_id'].unique())
        billing_codes.update(df_chunk['billing_code'].unique())
        
        # Create rate keys (provider|code|rate|type)
        rate_key_series = (
            df_chunk['provider_reference_id'].astype(str) + '|' +
            df_chunk['billing_code'].astype(str) + '|' +
            df_chunk['negotiated_rate'].astype(str) + '|' +
            df_chunk['billing_code_type'].astype(str)
        )
        rate_keys.update(rate_key_series.unique())
        
        if (i + 1) % 100 == 0 or (i + 1) == total_groups:
            print(f"  Processed {i+1}/{total_groups} row groups...")
    
    return provider_ids, rate_keys, billing_codes

# Process files ONE AT A TIME and save to disk
print("="*80)
print("EXTRACTING UNIQUE KEYS FROM ALL FILES (ONE AT A TIME)")
print("="*80)
print("Results will be saved to disk to avoid memory issues\n")

cache_dir = Path("output/uhc_comparison_cache")
cache_dir.mkdir(exist_ok=True)

file_metadata = {}
for name, path in existing_files.items():
    cache_file = cache_dir / f"{name.replace(' ', '_')}_keys.pkl"
    
    if cache_file.exists():
        print(f"✅ {name}: Loading from cache...")
        with open(cache_file, 'rb') as f:
            providers, rate_keys, billing_codes = pickle.load(f)
    else:
        pq_file = pq.ParquetFile(path)
        print(f"\n{name}:")
        print(f"  Rows: {pq_file.metadata.num_rows:,}")
        print(f"  Row groups: {pq_file.num_row_groups}")
        
        providers, rate_keys, billing_codes = extract_unique_keys(pq_file, name)
        
        # Save to disk immediately
        print(f"  💾 Saving to cache...")
        with open(cache_file, 'wb') as f:
            pickle.dump((providers, rate_keys, billing_codes), f)
    
    file_metadata[name] = {
        'providers_count': len(providers),
        'rate_keys_count': len(rate_keys),
        'billing_codes_count': len(billing_codes),
        'cache_file': cache_file
    }
    
    # Free memory - don't keep sets in memory
    del providers, rate_keys, billing_codes

print("\n" + "="*80)
print("✅ EXTRACTION COMPLETE - All results cached to disk")
print("="*80)
for name, meta in file_metadata.items():
    print(f"{name:20s}: {meta['providers_count']:,} providers, {meta['rate_keys_count']:,} rate keys, {meta['billing_codes_count']:,} codes")


EXTRACTING UNIQUE KEYS FROM ALL FILES (ONE AT A TIME)
Results will be saved to disk to avoid memory issues


IL Choice Plus:
  Rows: 220,863,665
  Row groups: 11044

📥 Streaming IL Choice Plus (processing ALL 11044 row groups)...
  Processed 100/11044 row groups...
  Processed 200/11044 row groups...
  Processed 300/11044 row groups...
  Processed 400/11044 row groups...
  Processed 500/11044 row groups...
  Processed 600/11044 row groups...
  Processed 700/11044 row groups...
  Processed 800/11044 row groups...
  Processed 900/11044 row groups...
  Processed 1000/11044 row groups...
  Processed 1100/11044 row groups...
  Processed 1200/11044 row groups...
  Processed 1300/11044 row groups...
  Processed 1400/11044 row groups...
  Processed 1500/11044 row groups...
  Processed 1600/11044 row groups...
  Processed 1700/11044 row groups...
  Processed 1800/11044 row groups...
  Processed 1900/11044 row groups...
  Processed 2000/11044 row groups...
  Processed 2100/11044 row groups...
  

In [3]:
# Build comparison matrix for rate keys (load from cache one pair at a time)
print("="*80)
print("RATE KEY OVERLAP MATRIX")
print("="*80)
print("\nShows what % of each file's rate keys exist in other files")
print("(Loading from cache one pair at a time to avoid memory issues)\n")

file_names = list(file_metadata.keys())
matrix = {}

# Calculate overlap percentages - load only 2 files at a time
for i, file1 in enumerate(file_names):
    matrix[file1] = {}
    
    # Load file1 keys
    with open(file_metadata[file1]['cache_file'], 'rb') as f:
        _, keys1, _ = pickle.load(f)
    
    for file2 in file_names:
        # Load file2 keys
        with open(file_metadata[file2]['cache_file'], 'rb') as f:
            _, keys2, _ = pickle.load(f)
        
        overlap = len(keys1 & keys2)
        pct = (overlap / len(keys1) * 100) if len(keys1) > 0 else 0
        matrix[file1][file2] = pct
        
        # Free file2 from memory
        del keys2
    
    # Free file1 from memory
    del keys1
    print(f"  Processed {i+1}/{len(file_names)} files...")

# Display matrix
print(f"\n{'File':<20s} | ", end="")
for name in file_names:
    print(f"{name[:12]:>12s} | ", end="")
print()
print("-" * (20 + 15 * len(file_names)))

for file1 in file_names:
    print(f"{file1:<20s} | ", end="")
    for file2 in file_names:
        pct = matrix[file1][file2]
        if pct == 100.0:
            print(f"{'100%':>12s} | ", end="")
        elif pct >= 90.0:
            print(f"{pct:>11.1f}% | ", end="")
        else:
            print(f"{pct:>11.1f}% | ", end="")
    print()


RATE KEY OVERLAP MATRIX

Shows what % of each file's rate keys exist in other files
(Loading from cache one pair at a time to avoid memory issues)

  Processed 1/7 files...
  Processed 2/7 files...
  Processed 3/7 files...
  Processed 4/7 files...
  Processed 5/7 files...
  Processed 6/7 files...
  Processed 7/7 files...

File                 | IL Choice Pl |  IL Core POS | NC Choice Pl | NY Choice EP | NY Choice Pl | GA Charter H | GA Choice HM | 
-----------------------------------------------------------------------------------------------------------------------------
IL Choice Plus       |         100% |         0.3% |         100% |         0.3% |         100% |         0.3% |         0.3% | 
IL Core POS          |         0.3% |         100% |         0.3% |         0.3% |         0.3% |         0.3% |         0.3% | 
NC Choice Plus       |        99.5% |         0.3% |         100% |         0.3% |        99.5% |         0.3% |         0.3% | 
NY Choice EPO        |         0.3

In [ ]:
# Identify subset relationships (100% overlap = subset)
# Use matrix data we already calculated (no need to reload)
print("="*80)
print("SUBSET RELATIONSHIPS")
print("="*80)
print("\nFiles that are 100% subsets of other files:\n")

subset_relationships = []
for file1 in file_names:
    for file2 in file_names:
        if file1 != file2:
            # Use matrix data (100% means file1 is subset of file2)
            if matrix[file1][file2] == 100.0:
                extra = file_metadata[file2]['rate_keys_count'] - file_metadata[file1]['rate_keys_count']
                subset_relationships.append((file1, file2, extra))

if subset_relationships:
    for subset, superset, extra in subset_relationships:
        print(f"✅ {subset:20s} is 100% subset of {superset:20s} (+{extra:,} extra rate keys)")
else:
    print("❌ No 100% subset relationships found")

# Find master file (file with most unique rate keys)
print("\n" + "="*80)
print("MASTER FILE IDENTIFICATION")
print("="*80)

file_sizes = [(name, meta['rate_keys_count']) for name, meta in file_metadata.items()]
file_sizes.sort(key=lambda x: x[1], reverse=True)

print("\nFiles ranked by number of unique rate keys:")
for i, (name, count) in enumerate(file_sizes, 1):
    marker = "👑" if i == 1 else "  "
    print(f"{marker} {i}. {name:20s}: {count:,} unique rate keys")


SUBSET RELATIONSHIPS

Files that are 100% subsets of other files:

✅ IL Choice Plus       is 100% subset of NC Choice Plus       (+621,353 extra rate keys)
✅ IL Choice Plus       is 100% subset of NY Choice Plus       (+22,457 extra rate keys)
✅ NY Choice Plus       is 100% subset of NC Choice Plus       (+598,896 extra rate keys)

MASTER FILE IDENTIFICATION

Files ranked by number of unique rate keys:
👑 1. NC Choice Plus      : 119,461,113 unique rate keys
   2. GA Choice HMO       : 119,357,919 unique rate keys
   3. NY Choice EPO       : 118,958,754 unique rate keys
   4. NY Choice Plus      : 118,862,217 unique rate keys
   5. IL Choice Plus      : 118,839,760 unique rate keys
   6. IL Core POS         : 112,288,723 unique rate keys
   7. GA Charter HMO      : 110,864,888 unique rate keys


: 

In [ ]:
# Calculate unique data per file (estimated from matrix - memory-efficient)
print("="*80)
print("UNIQUE DATA PER FILE")
print("="*80)
print("\nEstimated unique rate keys per file (using overlap matrix):\n")

# For each file, find the maximum overlap with any other file
# Unique keys ≈ total keys - max overlap with any other file
for name in file_names:
    total_keys = file_metadata[name]['rate_keys_count']
    
    # Find max overlap with any other file
    max_overlap_pct = 0
    max_overlap_file = None
    for other_name in file_names:
        if other_name != name:
            overlap_pct = matrix[name][other_name]
            if overlap_pct > max_overlap_pct:
                max_overlap_pct = overlap_pct
                max_overlap_file = other_name
    
    # Estimate unique keys (conservative: total - max overlap)
    max_overlap_count = int(total_keys * max_overlap_pct / 100)
    estimated_unique = total_keys - max_overlap_count
    unique_pct = (estimated_unique / total_keys * 100) if total_keys > 0 else 0
    
    print(f"{name:20s}: ~{estimated_unique:,} unique rate keys ({unique_pct:.2f}% of file)")
    if max_overlap_file and max_overlap_pct > 0.1:
        print(f"  {'':20s}  (max overlap: {max_overlap_pct:.1f}% with {max_overlap_file})")

print("\n" + "-"*80)
print("💡 Note: This is an estimate. True unique keys may be lower if")
print("   a key exists in multiple files (not just the max overlap file).")


UNIQUE DATA PER FILE

Rate keys that exist ONLY in each file (not in any other):

(This may take a few minutes - loading files one at a time)



In [ ]:
# Provider overlap analysis (using cached metadata only - no loading)
print("="*80)
print("PROVIDER OVERLAP ANALYSIS")
print("="*80)

# Find unique providers per file (from metadata)
print("\nUnique providers per file:")
provider_counts = {}
for name, meta in file_metadata.items():
    count = meta['providers_count']
    provider_counts[name] = count
    print(f"  {name:20s}: {count:,} providers")

# Find max providers (likely the master)
max_providers = max(provider_counts.values())
max_provider_file = max(provider_counts.items(), key=lambda x: x[1])[0]

print(f"\n📊 Maximum providers in single file: {max_providers:,} ({max_provider_file})")
print(f"   (This is likely the master provider network)")

# Estimate total unique providers (sum - rough estimate)
total_estimate = sum(provider_counts.values())
print(f"\n💡 Rough estimate of total unique providers: ~{total_estimate:,}")
print(f"   (Actual total may be lower due to overlap)")
print(f"   (To get exact count, would need to load all provider sets - memory intensive)")


In [ ]:
# Summary and recommendations (use cached metadata)
print("="*80)
print("SUMMARY & RECOMMENDATIONS")
print("="*80)

# Find master file (from metadata)
master_name = max(file_metadata.items(), key=lambda x: x[1]['rate_keys_count'])[0]
master_meta = file_metadata[master_name]

print(f"\n👑 MASTER FILE: {master_name}")
print(f"   Contains {master_meta['rate_keys_count']:,} unique rate keys")
print(f"   Contains {master_meta['providers_count']:,} unique providers")
print(f"   Contains {master_meta['billing_codes_count']:,} unique billing codes")

# Count how many files are subsets of master (from matrix)
subset_count = 0
for name in file_names:
    if name != master_name:
        if matrix[name][master_name] == 100.0:
            subset_count += 1

print(f"\n📊 {subset_count}/{len(file_metadata)-1} other files are 100% subsets of master")

print("\n💡 INTERPRETATION:")
if subset_count == len(file_metadata) - 1:
    print("   ⚠️  All files are subsets of master - likely same national network")
    print("   → Consider using master file only for analysis")
    print("   → Other files may differ only in metadata (state/plan labels)")
else:
    print(f"   ✅ {len(file_metadata) - subset_count - 1} files have unique data")
    print("   → Files represent distinct networks/plans")
    print("   → All files needed for complete analysis")

print("\n💾 Cache files saved to: output/uhc_comparison_cache/")
print("   (Delete this folder to re-extract from scratch)")
